# Setup

In [ ]:
import torch

from project.config import TARGET_EDGE

from project.model import MF as Model
from project.utils.loss import BPRLoss
from torch_geometric.loader import LinkNeighborLoader
from torch.optim import Adam

from functools import partial

from project.utils.train import dispatch_epoch, dispatch_session
from project.utils.score import composite, recall_score, ndcg_score
from project.model import mf as model

# Data Handling

In [ ]:
# Loads the datasets to compute the loss.
trn_data = torch.load('data/out/trn_Video_Games.pt')
vld_data = torch.load('data/out/vld_Video_Games.pt')

# Specifies the shared key-word arguments for the batch loaders.
base_kwargs = dict(
    num_neighbors=[0],  # [16, 16],
    num_workers=10,
    pin_memory=True
)

# Extracts the edge attribute indices for all datasets.
trn_edge_label_index = trn_data[TARGET_EDGE].edge_label_index
vld_edge_label_index = vld_data[TARGET_EDGE].edge_label_index

# Specifies the shared key-word arguments for the loss batch loaders.
loss_kwargs = dict(
    **base_kwargs,
    neg_sampling='triplet',
    shuffle=True
)
# Creates the sub-graph loaders for the loss .
trn_loader = LinkNeighborLoader(**loss_kwargs,
    data=trn_data,
    edge_label_index=[TARGET_EDGE, trn_edge_label_index],
    batch_size=2048
)
vld_loader = LinkNeighborLoader(**loss_kwargs,
    data=vld_data,
    edge_label_index=[TARGET_EDGE, vld_edge_label_index],
    batch_size=4096
)

# Builds the dataset for the all-ranking protocol.
rnk_data = torch.load('data/out/rnk_vld_Video_Games.pt')
# Extracts the edge label index for the all-ranking data.
rnk_edge_label_index = rnk_data[TARGET_EDGE].edge_label_index
rnk_edge_label = rnk_data[TARGET_EDGE].edge_label
# Creates a batch loader for the all-ranking data.
rnk_loader = LinkNeighborLoader(**base_kwargs,
    data=rnk_data,
    edge_label_index=[TARGET_EDGE, rnk_edge_label_index],
    edge_label=rnk_edge_label,
    batch_size=8192,
)

# Training

In [ ]:
# Creates the model to train.
# module = Model(
#     num_embeddings=trn_data.num_nodes_dict,
#     embedding_dims=[64] * (3 + 1)
# )
module = Model(
    num_embeddings=trn_data.num_nodes_dict,
    embedding_dim=64,
    weights=3
)
display(module)

# Instanciates the learning algorithm and loss criterion.
optimizer = Adam(module.parameters(), lr=1e-3)
loss_fn = BPRLoss(module.parameters(), reg_factor=1e-4)

# Builds the evaluation function.
batch_handler = partial(model.evaluate, edge_type=TARGET_EDGE)
# Builds the update and validate function.
update_fn = partial(dispatch_epoch,
    loader=trn_loader,
    loss_fn=loss_fn,
    optimizer=optimizer,
    batch_handler=batch_handler
)
validate_fn = partial(dispatch_epoch,
    loader=vld_loader,
    loss_fn=loss_fn,
    batch_handler=batch_handler
)

# Builds the score function.
score_fn = partial(composite,
    loader=rnk_loader,
    pred_fn=model.predict,
    edge_type=TARGET_EDGE,
    score_fns=[
        recall_score, 
        ndcg_score
    ],
    at_k=20
)

In [ ]:
dispatch_session(
    module=module,
    update_fn=update_fn,
    validate_fn=validate_fn,
    score_fn=score_fn,
    verbose=True,
    num_epochs=32,
    path='/home/jowin/Git/GNN-CF/out/light_gcn/1e-4'
)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

from pandas import DataFrame


# Setting the plotting style.
sns.set()

# Loading the data.
with open('/home/jowin/Git/GNN-CF/out/light_gcn/1e-6/trc.pkl', 'rb') as file:
    trace = pickle.load(file)

# Prepares the data for plotting.
score = DataFrame(trace['score'], columns=['Recall', 'NDCG']) \
    .mul(100) \
    .rename_axis(index='epoch', columns='metric') \
    .stack() \
    .rename('score') \
    .reset_index()

# Plots the score trace.
fig, ax = plt.subplots(figsize=[7, 3], dpi=192)
ax = sns.lineplot(
    data=score, 
    x='epoch', 
    y='score', 
    hue='metric', 
    style='metric', 
    ax=ax
)
ax.set_xlabel('Epoch')
ax.set_ylabel('Score@20 (%)')
ax.legend(title='Metric')
plt.show(fig)